# Import the packages

Before running your first cell, make sure GPU is enabled! Click the three dots in the upper right, go to 'Accelerator' and select 'GPU P100'

In [2]:
%matplotlib ipympl

import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from inception import Inception, InceptionBlock



# Load and preprocess the training data

In [3]:
# Loading the data files from Kaggle into this workspace
basal_train = np.load('basal_train.npy')
pre_seizure_train = np.load('pre_seizure_train.npy')

# Checking the dimensions of our data
print(basal_train.shape)
print(pre_seizure_train.shape)

# Original train data is 3x2x24000, 3 subjects, 2 electrodes and 240 seconds
# Below we will concatenate across subjects

# Reshape basal_train to a 2x72000 matrix (2 electrodes, 720 seconds of data)
basal_train_reshaped = basal_train.reshape((basal_train.shape[1], -1))
pre_seizure_train_reshaped = pre_seizure_train.reshape((pre_seizure_train.shape[1], -1))

# Check the dimensions of basal_train_reshaped
print(basal_train_reshaped.shape)
print(pre_seizure_train_reshaped.shape)

(3, 2, 240000)
(3, 2, 60000)
(2, 720000)
(2, 180000)


## Slice data into 2s segments

In [4]:
# Create a function to receive longer segments of data and divide it into smaller blocks

# The function receives the data variable, the size of the smaller blocks it will be divided into
# and the related training labels (which tell whether that block is pre-epileptic or not)
def create_windows_per_recording(data, window_size, label):
    # Initialize the variables
    windows = []
    labels = []
    
    # Compute the number of windows based on the length of the data and the size of the window
    num_windows = (data.shape[1] - window_size) // window_size + 1
    for i in range(num_windows):
        # Fill the windows with segments of the original data
        window = data[:, i * window_size : i * window_size + window_size]
        windows.append(window)
        labels.append(label)
        
    # Return the resulting smaller windows and its labels
    return np.array(windows), np.array(labels)

### Balance the labels 50/50

In [5]:
# Creates a function to subsample an imbalanced dataset and enforce a 50/50 label distribution

# The function receives your X and y train-validation datasets and returns a subsampled version of them
def balance_labels(X_train, X_val, y_train, y_val):
    # Calculate the number of pre-epileptic samples
    num_pre_epileptic_train = np.sum(y_train == 1)
    num_pre_epileptic_val = np.sum(y_val == 1)
    
    # Calculate the number of desired basal samples
    desired_num_basal_train = num_pre_epileptic_train
    desired_num_basal_val= num_pre_epileptic_val

    # Select the basal samples for the train set
    basal_train_indices = np.where(y_train == 0)[0]
    
    # Select the basal samples for the validation set
    basal_val_indices = np.where(y_val == 0)[0]

    # Randomly subsample the basal indices
    selected_basal_train_indices = np.random.choice(basal_train_indices, size=desired_num_basal_train, replace=False)
    selected_basal_val_indices = np.random.choice(basal_val_indices, size=desired_num_basal_val, replace=False)
    
    # Get the pre-epileptic sample indices
    selected_pre_epileptic_train_indices = np.where(y_train == 1)[0]
    selected_pre_epileptic_val_indices = np.where(y_val == 1)[0]

    # Combine the selected pre-epileptic and basal windows for the validation set
    selected_train_indices = np.concatenate([selected_pre_epileptic_train_indices, selected_basal_train_indices])
    
    # Combine the selected pre-epileptic and basal windows for the validation set
    selected_val_indices = np.concatenate([selected_pre_epileptic_val_indices, selected_basal_val_indices])

    # Update the training and validation sets
    X_val = X_val[selected_val_indices]
    y_val = y_val[selected_val_indices]

    # Update the training and validation sets
    X_train = X_train[selected_train_indices]
    y_train = y_train[selected_train_indices]

    return X_train, X_val, y_train, y_val


## Set parameters and split train-test data


In [6]:
# Parameters
window_size = 2000  # equivalent to two seconds of data, equal to the test samples

# Create windows
# Assign '0' to the basal (non pre-epileptic) data and '1' to pre-epileptic
basal_windows, basal_labels = create_windows_per_recording(basal_train_reshaped, window_size, 0) 
pre_seizure_windows, pre_seizure_labels = create_windows_per_recording(pre_seizure_train_reshaped, window_size, 1)

# Concatenate the training windows and their labels
X_train = np.concatenate([basal_windows, pre_seizure_windows])
y_train = np.concatenate([basal_labels, pre_seizure_labels])
# remove the singleton dimension
X_train = np.squeeze(X_train)
y_train = np.squeeze(y_train)

##### Data augmentations #####

# Artificially create "new" samples by slightly modifying the originals

# Noise
noise = np.random.normal(0, 0.05, X_train.shape)
aug_noise = X_train + noise

# Scaling
scaling_factor = np.random.uniform(0.5, 1.5)
aug_scale = X_train * scaling_factor

# Flipping
aug_flipped = X_train[:, ::-1]

###############################

# Append augmented data to original training dataset
X_train = np.concatenate([X_train, aug_noise, aug_scale, aug_flipped])
y_train = np.concatenate([y_train, y_train, y_train, y_train])


# Split data into training and validation sets 
# 'test_size' specifies % of data towards validation set, 0.2 = 20%
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=70)

# Balance the training and validation set labels
X_train, X_val, y_train, y_val = balance_labels(X_train, X_val, y_train, y_val)

In [ ]:
# Scaling the data 
# Scaling can help the model overcome changes in magnitude that do not contain information about the target
# However, if amplitude variations contain informations about the target, this will be detrimental to the model
""" scaler = RobustScaler()
X_train = X_train.reshape(-1, X_train.shape[-1])
X_train = scaler.fit_transform(X_train)
X_train = X_train.reshape(-1, 2, 2000)
X_val = X_val.reshape(-1, X_val.shape[-1])
X_val = scaler.transform(X_val)
X_val = X_val.reshape(-1, 2, 2000) """

In [8]:
# print the shapes of the data
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_val shape: ", X_val.shape)
print("y_val shape: ", y_val.shape)

# Print the number of pre-epileptic and basal windows in the training and validation sets
print("Number of pre-epileptic windows in the training set: ", np.sum(y_train == 1))
print("Number of basal windows in the training set: ", np.sum(y_train == 0))

print("Number of pre-epileptic windows in the validation set: ", np.sum(y_val == 1))
print("Number of basal windows in the validation set: ", np.sum(y_val == 0))

# Print the proportions of pre-epileptic and basal windows in the training and validation sets
print("Proportion of pre-epileptic windows in the training set: ", np.sum(y_train == 1) / len(y_train))
print("Proportion of basal windows in the training set: ", np.sum(y_train == 0) / len(y_train))

print("Proportion of pre-epileptic windows in the validation set: ", np.sum(y_val == 1) / len(y_val))
print("Proportion of basal windows in the validation set: ", np.sum(y_val == 0) / len(y_val))

X_train shape:  (608, 2, 2000)
y_train shape:  (608,)
X_val shape:  (112, 2, 2000)
y_val shape:  (112,)
Number of pre-epileptic windows in the training set:  304
Number of basal windows in the training set:  304
Number of pre-epileptic windows in the validation set:  56
Number of basal windows in the validation set:  56
Proportion of pre-epileptic windows in the training set:  0.5
Proportion of basal windows in the training set:  0.5
Proportion of pre-epileptic windows in the validation set:  0.5
Proportion of basal windows in the validation set:  0.5


## Transform the data into Pytorch datasets and loaders

In [9]:
# Convert data to PyTorch tensors (a data structure)
tensor_x_train = torch.Tensor(X_train) 
tensor_x_val = torch.Tensor(X_val) 
tensor_y_train = torch.Tensor(y_train)
tensor_y_val = torch.Tensor(y_val)

# Create Tensor datasets
train_data = TensorDataset(tensor_x_train, tensor_y_train)
val_data = TensorDataset(tensor_x_val, tensor_y_val)

# Dataloaders (to feed the data into the model during training)
# 'batch_size' defines the chunk size of data to be fed in each training step
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size, drop_last=True)


In [10]:
print(tensor_x_val.shape,tensor_y_val.shape)
print(tensor_x_train.shape,tensor_y_train.shape)

torch.Size([112, 2, 2000]) torch.Size([112])
torch.Size([608, 2, 2000]) torch.Size([608])


### Check if the dataloader is functioning properly

In [11]:
train_iter = iter(train_loader)
try:
    data, label = next(train_iter)
except StopIteration:
    print("The train_loader is empty.")
    
val_iter = iter(val_loader)
try:
    data, label = next(val_iter)
except StopIteration:
    print("The val_loader is empty.")

# Build the model

## InceptionTime model

In [12]:
class Flatten(nn.Module):
	def __init__(self, out_features):
		super(Flatten, self).__init__()
		self.output_dim = out_features

	def forward(self, x):
		return x.view(-1, self.output_dim)
    
class Reshape(nn.Module):
	def __init__(self, out_shape):
		super(Reshape, self).__init__()
		self.out_shape = out_shape

	def forward(self, x):
		return x.view(-1, *self.out_shape)

InceptionTime = nn.Sequential(
                    Reshape(out_shape=(2,2000)),
                    InceptionBlock(
                        in_channels=2, 
                        n_filters=32, 
                        kernel_sizes=[5, 11, 23],
                        bottleneck_channels=32,
                        use_residual=True,
                        activation=nn.ReLU()
                    ),
                    nn.Dropout(0.2),
                    InceptionBlock(
                        in_channels=32*4, 
                        n_filters=32, 
                        kernel_sizes=[5, 11, 23],
                        bottleneck_channels=32,
                        use_residual=True,
                        activation=nn.ReLU()
                    ),
                    nn.Dropout(0.2),
                    nn.AdaptiveAvgPool1d(output_size=1),
                    Flatten(out_features=32*4*1),
                    nn.Linear(in_features=4*32*1, out_features=1),
        )

## Train the model

In [16]:
# InceptionTime model

# Loss and optimization
# 'device' sets which computing unit will be used, GPU(cuda) is preferred,
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 'criterion' specifies the method to compute loss (an 'error' metric for model prediction)
#criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.BCEWithLogitsLoss().to(device)#BCEWithLogitsLoss is the same as BCE, but with a sigmoid layer added before the BCE

# 'optimizer' sets the algorithm that is used to adjust the weights of the model
# Here we use ADAM, which is a fairly popular optimization algo
# 'lr' means Learning Rate, which modulates how fast the weights are adjusted
optimizer = torch.optim.Adam(InceptionTime.parameters(), lr=0.001, weight_decay=0.01) #weight_decay is a regularization parameter

# .to(device) simply means whe are directing this to the GPU or CPU (chosen above)
InceptionTime = InceptionTime.to(device)
threshold = 0 

""" # Load the model from a saved point if exists
try:
    InceptionTime.load_state_dict(torch.load('best_model.pth'))
    print("Model loaded successfully")
except FileNotFoundError:
    print("No saved model found, starting training from scratch")
 """
 
 # Initialize training losses, validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []



####### Training loop #######

# min_val_loss will be used to save the model with best performance (lowest loss)
min_val_loss = np.inf
# patience defines how many epochs without improvement will be tolerated before early stopping
patience = 150
patience_counter = 0

# Training loop
# epoch is each training step
for epoch in range(300):
    # initialize single evaluation metrics (for this iteration)
    running_loss = 0.0
    running_accuracy = 0.0
    # receive inputs and labels from the training dataloader
    for i, data in enumerate(train_loader, 0):
        # send inputs and labels to device
        inputs, labels = data[0].to(device), data[1].to(device)
        # clear out the gradients of all parameters that the optimizer is tracking
        optimizer.zero_grad()
        # make model predictions
        outputs = InceptionTime(inputs).squeeze()
        # compute training loss based on predicted values
        loss = criterion(outputs, labels)
        # perform back propagation
        loss.backward()
        # apply optimizer to adjust the weights
        optimizer.step()
        
        # compute accuracy and store this epoch's loss
        predicted = outputs.detach().cpu().numpy()
        running_accuracy += accuracy_score(labels.cpu().numpy(), (predicted > threshold).astype(int))
        running_loss += loss.item()

    # Validation loss computation
    val_running_loss = 0.0
    val_running_accuracy = 0.0
    # receive inputs and labels from the validation dataloader
    for i, data in enumerate(val_loader, 0):
        # send inputs and labels to device
        inputs, labels = data[0].to(device), data[1].to(device)
        # make model predictions
        outputs = InceptionTime(inputs).squeeze()
        
        # compute accuracy and store this epoch's loss
        predicted = outputs.detach().cpu().numpy()
        val_running_accuracy += accuracy_score(labels.cpu().numpy(), (predicted > threshold).astype(int))
        val_loss = criterion(outputs, labels)
        val_running_loss += val_loss.item()
        
    # accuracy and loss are computed based on the size of the batch (e.g. % of correct outputs)    
    train_accuracies.append(running_accuracy / len(train_loader))
    val_accuracies.append(val_running_accuracy / len(val_loader))
    train_losses.append(running_loss / len(train_loader))
    val_losses.append(val_running_loss / len(val_loader))
    
    # save the model if it has the best performance so far
    if val_loss < min_val_loss:
        # Save the model
        torch.save(InceptionTime.state_dict(), 'best_model.pth')
        min_val_loss = val_loss

        patience_counter = 0  # Reset the patience counter
    else:
        # Increment the patience counter
        patience_counter += 1
# Check if we've run out of patience
    if patience_counter >= patience:
        print("Early stopping...")
        break
        
    # Save the model every 50 epochs
    if epoch % 50 == 0:
        torch.save(InceptionTime.state_dict(), f'InceptionTime_model_epoch_{epoch}.pth')

    

    print(f'Epoch {epoch+1}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_running_loss / len(val_loader)}')
    print(f'Epoch {epoch+1}, Training Accuracy: {running_accuracy / len(train_loader)}, Validation Accuracy: {val_running_accuracy / len(val_loader)}')


print('Finished Training')

Epoch 1, Training Loss: 0.6713345845540365, Validation Loss: 0.5650309324264526
Epoch 1, Training Accuracy: 0.5659722222222222, Validation Accuracy: 0.71875
Epoch 2, Training Loss: 0.6123920546637641, Validation Loss: 0.5178772211074829
Epoch 2, Training Accuracy: 0.6649305555555556, Validation Accuracy: 0.734375
Epoch 3, Training Loss: 0.5659895804193285, Validation Loss: 0.5403642654418945
Epoch 3, Training Accuracy: 0.6979166666666666, Validation Accuracy: 0.640625
Epoch 4, Training Loss: 0.5684597790241241, Validation Loss: 0.49397292733192444
Epoch 4, Training Accuracy: 0.6979166666666666, Validation Accuracy: 0.8125
Epoch 5, Training Loss: 0.5353647271792094, Validation Loss: 0.6123940944671631
Epoch 5, Training Accuracy: 0.7326388888888888, Validation Accuracy: 0.671875
Epoch 6, Training Loss: 0.5531839032967886, Validation Loss: 0.43642809987068176
Epoch 6, Training Accuracy: 0.7152777777777778, Validation Accuracy: 0.8125
Epoch 7, Training Loss: 0.511851128604677, Validation L

In [ ]:
# Save the InceptionTime model
#torch.save(InceptionTime.state_dict(), 'InceptionTime_model_80%Train.pth')

# Load the model from a file
#InceptionTime.load_state_dict(torch.load('InceptionTime_model_epoch_250.pth'))

# CPU-only machine with GPU-trained model
#InceptionTime.load_state_dict(torch.load('InceptionTime_model_epoch_300.pth',map_location=torch.device('cpu')))

# Plotting model performance

In [ ]:
# Plot the results
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(2, 2, 2)
plt.plot(train_accuracies, label='Training accuracy')
plt.plot(val_accuracies, label='Validation accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show()


# Create submission file using model predictions

In [20]:
# Test the model on the validation dataset

InceptionTime.eval()

with torch.no_grad():
        y_pred = InceptionTime(tensor_x_val.to(device)).cpu().numpy()

### Test the accuracy of model predictions in the validation dataset

In [42]:
#print(y_pred)
print(y_pred.shape)

# assign 1 when y_pred > 0 and 0 otherwise
predictions = (y_pred > -1).astype(int)

print(len(predictions[predictions>0]))

print(f1_score(y_val, predictions))
print(accuracy_score(y_val, predictions))

(112, 1)
55
0.972972972972973
0.9732142857142857


In [46]:
# Load the test set
X_test = np.load('test_set.npy')


# Convert to tensor
tensor_x_test = torch.Tensor(X_test)


# Get model predictions
InceptionTime.eval()
with torch.no_grad():
    y_pred = InceptionTime(tensor_x_test.to(device)).cpu().numpy()


predictions = (y_pred > -1).astype(int).squeeze()
#predictions = np.argmax(y_pred, axis=1)
print(len(predictions[predictions>0]))

4


In [69]:
predictions = (y_pred > -3.9).astype(int).squeeze()

# Create submission file
with open("submission.csv", "w") as f:
    f.write("win_id,label\n")
    for i, pred in enumerate(predictions):
        f.write(f"{i},{pred}\n")
        
print(len(predictions[predictions>0]))

124


In [40]:
# Plot the average time series of each channel (2nd dimension) of tensor_x_val and tensor_x_test

# Convert tensors to numpy arrays
tensor_x_val_plt = tensor_x_val.numpy()
tensor_x_test_plt = tensor_x_test.numpy()

# Using a subplot for each channel
plt.figure(figsize=(12,6))
plt.subplot(2, 2, 1)
plt.title('M1 electrode')
plt.plot(np.squeeze(np.mean(tensor_x_val_plt[:,0,:], axis=0)), label='tensor_x_val')
plt.plot(np.squeeze(np.mean(tensor_x_test_plt[:,0,:], axis=0)), label='tensor_x_test')
plt.legend()
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.subplot(2, 2, 2)
plt.title('VA electrode')
plt.plot(np.squeeze(np.mean(tensor_x_val_plt[:,1,:], axis=0)), label='tensor_x_val')
plt.plot(np.squeeze(np.mean(tensor_x_test_plt[:,1,:], axis=0)), label='tensor_x_test')
plt.legend()
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …